In [18]:
!uv pip install semsimian pandas tqdm

Resolved 8 packages in 166ms                                                 
Downloaded 1 package in 33ms                                        
Installed 1 package in 6ms5                                         
 + tqdm==4.66.5


In [3]:
from semsimian import Semsimian

predicates= [
    "rdfs:subClassOf",
    "BFO:0000050",
    "UPHENO:0000001",
]
semsimian = Semsimian(
    spo=None,
    predicates=predicates,
    pairwise_similarity_attributes=None,
    resource_path="./go.db",
)

In [6]:
def search(go_terms: list[str]):
    
    subject_prefixes = ["HGNC:"]
    assoc_predicate = {
        "biolink:enables",
        "biolink:actively_involved_in",
        "biolink:located_in",
        "biolink:active_in",
        "biolink:part_of",
        "biolink:acts_upstream_of",
        "biolink:contributes_to",
        "biolink:acts_upstream_of_or_within",
        "biolink:colocalizes_with",
        "biolink:acts_upstream_of_positive_effect",
        "biolink:acts_upstream_of_or_within_positive_effect",
        "biolink:acts_upstream_of_negative_effect",
        "biolink:acts_upstream_of_or_within_negative_effect"
    }
    limit = 50000
    return semsimian.associations_search(
        assoc_predicate, # assoc_predicate
        set(go_terms), # object_terms
        False, # is_subject    
        "full", # search_type
        None, # subject_terms
        subject_prefixes,
        "ancestor_information_content", # metric
        limit,
        "bidirectional" # direction
    )


In [7]:
import pandas as pd

def load_go_associations(file_path):
    # Read the TSV file into a DataFrame, skipping the first row
    df = pd.read_csv(file_path, delimiter='\t', skiprows=1, header=None)

    # Create a dictionary from the DataFrame
    result = {row[0]: row[1].split('|') for _, row in df.iterrows()}

    return result

go_associations = load_go_associations('go_list_per_gene.tsv')

In [ ]:
gencc = pd.read_csv('gencc.tsv', delimiter='\t')
gencc_genes = gencc['gene_curie'].unique().tolist()

# write to tsv file
with open('gencc_semsimian_search_results.tsv', 'w') as f:
    f.write('gene_a\tgene_b\tscore\n')

    for gene in gencc_genes:
        if gene not in go_associations:
            continue
        results = search(go_associations[gene])
        for result in results:
            matching_gene = result[2]
            score = result[0]
            f.write(f"{gene}\t{matching_gene}\t{score}\n")
        f.flush()#%%

gencc = pd.read_csv('gencc.tsv', delimiter='\t')
gencc_genes = gencc['gene_curie'].unique().tolist()

# write to tsv file
with open('gencc_semsimian_search_results.tsv', 'w') as f:
    f.write('gene_a\tgene_b\tscore\n')

    for gene in gencc_genes:
            results = search(go_associations[gene])            
            for result in results:
                matching_gene = result[2]
                score = result[0]
                f.write(f"{gene}\t{matching_gene}\t{score}\n")
            f.flush()

